# 1. Introducción

El siguiente cuaderno realiza la encriptación de un mensaje mediante un algoritmo de cifrado llamado "Cifrado de Transposición en columnas" ( en inglés: Columnar Transposition Cipher).

### Características

*   El mensaje se escribe en una matriz por filas de una longitud fija y luego se vuelve a leer columna por columna.
*   El ancho de las filas se definen mediante una palabra clave.

    **Ejemplo:** Si tomamos la palabra AUTO como clave, al tener 4 caracteres el largo de las filas es 4.

*   Los espacios que sobran en la matriz se rellenaron con el caracter  "_".

El objetivo de este ejercicio es dar un ejemplo de la optimización que podemos llegar a tener utilizando programación paralela con CUDA utilizando 2 dimensiones.

**Aclaración:** Solo se ejecuta de manera paralela con CUDA la formacion del lenguaje encriptado donde se reemplazan dos ciclos for anidados por la ejecucion paralela en dos dimensiones.


# 2. Armado del ambiente

*   Se instala el modulo de CUDA de Python en el cuaderno
*   Se carga un archivo de texto que se podra encriptar
*   Se importan las bibliotecas necesarias



In [5]:
#Se instala el modulo de CUDA
!pip install pycuda

#Se obtiene el texto desde el repo de git
url_texto="https://github.com/FranciscoPretto/EA2/blob/entrega/HPC/TextoSinEncriptar.txt?raw=true"
!wget {url_texto} -O sinEncriptar.txt

#Muestro el archivo obtenido
!cat sinEncriptar.txt

#Se importan las bibliotecas

from datetime import datetime
import matplotlib.pyplot as plt
import math 
import numpy
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule



--2020-12-02 20:55:43--  https://github.com/FranciscoPretto/EA2/blob/entrega/HPC/TextoSinEncriptar.txt?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/FranciscoPretto/EA2/raw/entrega/HPC/TextoSinEncriptar.txt [following]
--2020-12-02 20:55:43--  https://github.com/FranciscoPretto/EA2/raw/entrega/HPC/TextoSinEncriptar.txt
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FranciscoPretto/EA2/entrega/HPC/TextoSinEncriptar.txt [following]
--2020-12-02 20:55:44--  https://raw.githubusercontent.com/FranciscoPretto/EA2/entrega/HPC/TextoSinEncriptar.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.

# 3. Desarrollo




##3.1 Ejecución Secuencial

In [11]:
# --------------------------------------------
#@title 3.1.1 Parámetros de ejecución { vertical-output: true }


key =   "SISOP"#@param {type:"string"}
if(len(key)<1):
  raise Exception("La key debe contener al menos 1 caracter")

Encriptar_mensaje_propio = False #@param {type:"boolean"}
Mensaje = "" #@param{type:"string"}
if(Encriptar_mensaje_propio and len(Mensaje)<1):
    raise Exception("El mensaje debe contener al menos 1 caracter")
# --------------------------------------------
#Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
tiempo_total_secuencial  = datetime.now()

def deTxtaString():
  return open('sinEncriptar.txt', 'r').read()

def encriptarMensajeSecuencial(): 
    
    msg=""
    if(Encriptar_mensaje_propio):
      msg=Mensaje
    else:
      msg=deTxtaString()
    cipher = "" 
   
    msg_len = float(len(msg)) 
    msg_lst = list(msg) 
    key_lst = sorted(list(key)) 
    
    # Calcular tamaño de columna
    col = len(key) 
      
    # Calcular tamaño de fila
    row = int(math.ceil(msg_len / col)) 

  
    #Agregar caracter de relleno
    fill_null = int((row * col) - msg_len) 
    msg_lst.extend('_' * fill_null) 

  
    #Crear la matriz
    matrix = [msg_lst[i: i + col]  
              for i in range(0, len(msg_lst), col)] 
  
    # Leer la matriz y realizar el encriptado

    tiempo_cpu = datetime.now()

    for i in range(col):      
        for row in matrix:
          cipher +=row[i]                     
    
    tiempo_cpu = datetime.now() - tiempo_cpu
    tiempo_total_cpu = datetime.now() - tiempo_total_secuencial

    print("Tiempo Total: ", tiempo_en_ms( tiempo_total_cpu ), "[ms]" )
    print("Tiempo CPU: ", tiempo_en_ms( tiempo_cpu   ), "[ms]" )

    return cipher

# Decryption 
def desencriptar(cipher): 
    msg = "" 

    k_indx = 0
  
    msg_indx = 0
    msg_len = float(len(cipher)) 
    msg_lst = list(cipher) 
  
    col = len(key)      

    fila = int(math.ceil(msg_len / col)) 

    mensajeDescifrado = [] 
    for _ in range(fila): 
        mensajeDescifrado += [[None] * col] 
  

    for i in range(col): 
        for j in range(fila): 
            mensajeDescifrado[j][i] = msg_lst[msg_indx] 
            msg_indx += 1
        k_indx += 1

    msg = ''.join(sum(mensajeDescifrado, [])) 
   
    return msg 

resultado_encriptado_secuencial=encriptarMensajeSecuencial()
resultado_desencriptado_secuencial=desencriptar(resultado_encriptado_secuencial)


Tiempo Total:  4.842 [ms]
Tiempo CPU:  2.574 [ms]


##3.2 Ejecución paralela con CUDA

In [10]:
# --------------------------------------------
#@title 3.2.1 Parámetros de ejecución { vertical-output: true }


key =   "SISOP"#@param {type:"string"}
if(len(key)<1):
  raise Exception("La key debe contener al menos 1 caracter")

Encriptar_mensaje_propio = False #@param {type:"boolean"}
Mensaje = "" #@param{type:"string"}
if(Encriptar_mensaje_propio and len(Mensaje)<1):
    raise Exception("El mensaje debe contener al menos 1 caracter")
# --------------------------------------------

#CPU - Tomo el tiempo inicial total
tiempo_total_cuda = datetime.now()

#Encriptar


def deTxtaString():
  return open('sinEncriptar.txt', 'r').read()


def encriptarMensajeCUDA():

    msg=""

    #CPU - Evaluo si debo tomar el mensaje del parametro o el descargado
    
    if(Encriptar_mensaje_propio):    
      msg=Mensaje
    #CPU - Se abre el archivo previamente descargado
    else:
      msg=deTxtaString()

    mensaje_cifrado = ""

    #CPU - Transformo el string en lista
    
    try:
      msg_len = float(len(msg))
      if(msg_len is 0):
        raise ValueError("Cadena vacia")
    except ValueError:
        print("El mensaje debe tener al menos un caracter")
    msg_list = list(msg) 
 
    #CPU - Calculo numero de columnas
    col = len(key)

    #CPU - Calculo máximo numero de filas
    fila = int(math.ceil(msg_len / col)) 

    #CPU - Agregar los caracteres de relleno en los lugares sobrantesd de la matriz

    fill_null = int((fila * col) - msg_len) 
    msg_list.extend('_' * fill_null)

    #CPU - Crear la matriz e insertar los datos

    matriz = [msg_list[i: i + col]  
              for i in range(0, len(msg_list), col)] 
    
    # CPU - Defino la memoria de los vectores en cpu.

    matriz_cpu = numpy.asmatrix(matriz)
    vector_respuesta_cpu = numpy.empty_like(msg_list)

    # GPU - reservo la memoria GPU.

    matriz_gpu = cuda.mem_alloc(matriz_cpu.nbytes)
    vector_respuesta_gpu = cuda.mem_alloc(vector_respuesta_cpu.nbytes)

    # GPU - Copio la memoria al GPU.
    cuda.memcpy_htod(matriz_gpu,matriz_cpu)

    # CPU - Defino la función kernel que ejecutará en GPU.
    module = SourceModule("""
    __global__ void transposition_cipher( int filas, int columnas, int* matriz, int* resultado)   
    {
        // Calculo las coordenadas del Thread en dos dimensiones.
        int idx = threadIdx.x + blockIdx.x*blockDim.x;
        int idy = threadIdx.y + blockIdx.y*blockDim.y;     
          
          if(idx < columnas && idy <  filas)
          {
              int pos_res =(idx*filas)+idy;            
              int pos_matriz =(idy*columnas)+idx;
              resultado[pos_res] = matriz[pos_matriz];
             
      }

    }
        
    """)

    # CPU - Genero la función kernel.
    kernel = module.get_function("transposition_cipher")

    #CPU - Tomo el tiempo inicial de ejecución de la función kernel.
    tiempo_gpu = datetime.now()

    # GPU - Ejecuta el kernel.
    dim_hilo_x = 19 
    dim_bloque_x = numpy.int( (col+dim_hilo_x-1) / dim_hilo_x )

    dim_hilo_y = 16
    dim_bloque_y = numpy.int( (fila+dim_hilo_y-1) / dim_hilo_y )

    kernel( numpy.int32(fila), numpy.int32(col),matriz_gpu,vector_respuesta_gpu,block=( dim_hilo_x, dim_hilo_y, 1 ), grid=(dim_bloque_x, dim_bloque_y,1) )

    #CPU - Tomo el tiempo final de ejecución de la función kernel.
    tiempo_gpu = datetime.now() - tiempo_gpu

    # GPU - Copio el resultado desde la memoria GPU.
    cuda.memcpy_dtoh( vector_respuesta_cpu, vector_respuesta_gpu )

    #CPU- Tomo el tiempo total final.
    tiempo_total = datetime.now() - tiempo_total_cuda

    #CPU - Informo los resultados.
    print( "Thread: [", dim_hilo_x, ",", dim_hilo_y, " ], Bloque : [", dim_bloque_x, ",", dim_bloque_y, "]" )
    print( "Total de Thread: [", dim_hilo_x*dim_bloque_x, ",", dim_hilo_y*dim_bloque_y, " ]", " = ", dim_hilo_x*dim_bloque_x*dim_hilo_y*dim_bloque_y )
    print("Tiempo Total: ", tiempo_en_ms( tiempo_total ), "[ms]" )
    print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )

    return vector_respuesta_cpu

    

# Desencriptar Mensaje ( Se realiza completamente en CPU)
def desencriptarMensaje(vector_cpu): 
    
    msg = "" 
  
    # Generar lista del mensaje cifrado
    msg_indx = 0
    msg_list = vector_cpu.tolist()
    msg_len = len(msg_list) 
    
    # Calcular la columna de la matriz
    col = len(key) 
      
    # Calcular el máximo de filas
    fila = int(math.ceil(msg_len/ col)) 
  
  
   # Crear la matriz para guardar mensaje descifrado
    mensajeDescifrado = [] 

    for _ in range(fila): 
        mensajeDescifrado += [[None] * col] 
  
  #Leer el mensaje cifrado y colocarlo en la matriz correspondiente
    for i in range(col): 
        for j in range(fila): 
            mensajeDescifrado[j][i] = msg_list[msg_indx] 
            msg_indx += 1
  
    # Convertir la matriz en un string
    msg = ''.join(sum(mensajeDescifrado, [])) 
 
    return msg 

resultado_encriptado_paralelo=encriptarMensajeCUDA()
resultado_descencriptado_paralelo=desencriptarMensaje(resultado_encriptado_paralelo)


Thread: [ 19 , 16  ], Bloque : [ 1 , 409 ]
Total de Thread: [ 19 , 6544  ]  =  124336
Tiempo Total:  825.904 [ms]
Tiempo GPU:  2.169 [ms]


# 4. Tabla de pasos

> Se define la tabla de pasos del punto 3.2


Procesador | Función | Detalle
------------|---------|----------
CPU      |  !wget {url_texto} -O sinEncriptar.txt       | Lectura de la dirección URL para obtener texto.
CPU      | pip install pycuda    | Instala en el cuaderno los driver de CUDA para Python.
CPU      |  @param                | Lectura del tamaño de paramtros desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  if(Encriptar_mensaje_propio):            | Evaluo si debo tomar el mensaje del parametro o el descargado
CPU      | deTxtaString()|Se abre el archivo previamente descargado
CPU      | list(msg)|Transformo el string en lista
CPU      | len(key)|Calculo numero de columnas
CPU      | int(math.ceil(msg_len / col)) |Calculo máximo numero de filas
CPU      |  fill_null = int((fila * col) - msg_len);  msg_list.extend('_' * fill_null)|Agregar los caracteres de relleno en los lugares sobrantes de la matriz
CPU      | matriz = [msg_list[i: i + col]  for i in range(0, len(msg_list), col)] |Crear la matriz e insertar los datos
CPU      |  numpy.asmatrix()  | Defino la memoria de los vectores en cpu. 
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria para las imagenes en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copio los valores en crudo de las imagenes al GPU.
CPU      |  SourceModule()        | Defino la función kernel
CPU      |  module.get_function() | Convierte el texto del kernel en funcion de Python.
CPU      |  dim_hilo_x, dim_hilo_y| Calcula las dimensiones para la ejecuciòn de 2D.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU, enviando los parametros.
CPU      | cuda.memcpy_dtoh()     | Copia desde la memoria GPU al CPU.
CPU      |  print()               | Informa los atributos de la imagen.
CPU      | desencriptarMensaje()  | Desencripta el mensaje para ver la eficacia del algoritmo

# 5. Conclusiones

Se ejecutaron 10 veces ambos algoritmos y se obtuvio el siguiente promedio:

CPU - 3.1:
*   Tiempo Total:  7.312 [ms]
*   Tiempo CPU:  2.725 [ms]

GPU - 3.2:

*   Tiempo Total:  9.16 [ms]
*   Tiempo GPU:  0.06 [ms]

Se puede apreciar que, habiendo usado como imput el archivo previamente descargado, la optimizacion en la ejecucion paralela supero por mas de 47 veces el tiempo secuencial pero igualmente, al ser este tan chico, la preparación y ejecuciones previas que debemos realizar para enviar el codigo fuente a la GPU empeoraron el algoritmo. El archivo utilizado tiene aproximadamente 75000 caracteres, para que el algoritmo paralelo sea optimo se deben procesar mas datos.

Se pueden extraer tres grandes algoritmos en este cuaderno, los que realizan la encripción del mensaje, uno de forma secuencial con un for anidado y otro de forma paralela utilizando CUDA en 2 dimensiones y el algoritmo que decifra el mensaje que en este caso es secuencial y el mismo en ambos puntos. 

La forma como trabaja las dimensiones CUDA es muy interesante y en mi caso, confusa al comienzo. El no tener una matriz dentro de la ejecución cuda, sino un vector generado desde una matriz me trajo bastante trabajo, pero luego de analizar los ejemplos de la catedra y con prueba y error lo pude razonar.

Este ejercicio me llevo a pensar las optimizaciones que uno podria colocar en los equipos tienen el disco encriptando los datos constantemente, se podria reducir el consumo del CPU considerablemente y aun asi tener la encriptación activa.

La evolución natural del ejercico es realizar el algoritmo de descifrado paralelo y comparar como afecta a la performance ese cambio. Asumo que sera similar al otro algoritmo.


#6. Bibliografía

Se utilizaron los apuntes de la catedra y las siguientes páginas web:


*   https://www.tutorialspoint.com/cryptography_with_python/cryptography_with_python_transposition_cipher.htm
*   https://thales.cica.es/rd/Recursos/rd99/ed99-0289-02/ed99-0289-02.html

